<a href="https://colab.research.google.com/github/VinithaRao22/tf-vs-pytorch-lab/blob/main/Lab03_TensorFlow_vs_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        # Fill input shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Fill number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Fill number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Fill name of loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Output training time
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8618 - loss: 0.4941
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9553 - loss: 0.1545
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9687 - loss: 0.1087
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9759 - loss: 0.0809
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9804 - loss: 0.0677
TF Training time: 33.46 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9661 - loss: 0.1088


[0.09457345306873322, 0.9710999727249146]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp527a9i4f'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134940173694160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134940173696656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134940173696848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134940173695888: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Fill correct input and output size
        self.fc2 = nn.Linear(64, 10)    # Fill correct input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))    # Fill correct layer
        return self.fc2(x)    # Fill correct layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

PyTorch Training time: 64.67 seconds
Test accuracy: 0.9633


## Convert PyTorch model to ONNX

In [5]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 88.3 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.4497, Accuracy: 0.1250
Step 100, Loss: 0.3338, Accuracy: 0.7160
Step 200, Loss: 0.2329, Accuracy: 0.7956
Step 300, Loss: 0.2287, Accuracy: 0.8275
Step 400, Loss: 0.1954, Accuracy: 0.8476
Step 500, Loss: 0.5050, Accuracy: 0.8595
Step 600, Loss: 0.2571, Accuracy: 0.8697
Step 700, Loss: 0.2180, Accuracy: 0.8762
Step 800, Loss: 0.5224, Accuracy: 0.8820
Step 900, Loss: 0.5155, Accuracy: 0.8870
Step 1000, Loss: 0.1557, Accuracy: 0.8916
Step 1100, Loss: 0.1322, Accuracy: 0.8961
Step 1200, Loss: 0.2209, Accuracy: 0.8989
Step 1300, Loss: 0.4585, Accuracy: 0.9021
Step 1400, Loss: 0.1922, Accuracy: 0.9042
Step 1500, Loss: 0.2942, Accuracy: 0.9068
Step 1600, Loss: 0.2936, Accuracy: 0.9097
Step 1700, Loss: 0.3227, Accuracy: 0.9119
Step 1800, Loss: 0.0912, Accuracy: 0.9140
Training Accuracy for epoch 1: 0.9153

Epoch 2/5
Step 0, Loss: 0.1152, Accuracy: 0.9688
Step 100, Loss: 0.1537, Accuracy: 0.9502
Step 200, Loss: 0.3175, Accuracy: 0.9524
Step 300, Loss: 0.1929, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [9]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3801, Accuracy: 0.1250
Step 100, Loss: 0.5791, Accuracy: 0.7287
Step 200, Loss: 0.6438, Accuracy: 0.8004
Step 300, Loss: 0.5098, Accuracy: 0.8339
Step 400, Loss: 0.6336, Accuracy: 0.8527
Step 500, Loss: 0.2519, Accuracy: 0.8640
Step 600, Loss: 0.5093, Accuracy: 0.8739
Step 700, Loss: 0.1533, Accuracy: 0.8817
Step 800, Loss: 0.1979, Accuracy: 0.8867
Step 900, Loss: 0.6330, Accuracy: 0.8909
Step 1000, Loss: 0.2887, Accuracy: 0.8946
Step 1100, Loss: 0.4244, Accuracy: 0.8983
Step 1200, Loss: 0.2061, Accuracy: 0.9010
Step 1300, Loss: 0.0990, Accuracy: 0.9028
Step 1400, Loss: 0.3325, Accuracy: 0.9052
Step 1500, Loss: 0.1648, Accuracy: 0.9076
Step 1600, Loss: 0.2779, Accuracy: 0.9096
Step 1700, Loss: 0.1347, Accuracy: 0.9120
Step 1800, Loss: 0.1129, Accuracy: 0.9143
Training Accuracy for epoch 1: 0.9160

Epoch 2/5
Step 0, Loss: 0.2344, Accuracy: 0.9375
Step 100, Loss: 0.0783, Accuracy: 0.9530
Step 200, Loss: 0.4860, Accuracy: 0.9548
Step 300, Loss: 0.0547, Accuracy: